In [1]:
! conda install numpy pandas matplotlib seaborn -y
! conda install nltk -y
! conda install scikit-learn -y

import numpy as np
import pandas as pd
import seaborn as sns

Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.

Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.

Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.



# Alternative Approach with LSTM

In [1]:
from gensim.models import Word2Vec

### Remove Stopwords
turned off, as train accuracy was only at arround 60% 

In [2]:
remove_stopwords = False

In [3]:
import pandas as pd
df = pd.read_csv('OnionOrNot.csv')

# stop words
! conda install nltk -y
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
stop = stopwords.words('english')
# remove stopwords
if remove_stopwords:
    df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df.head()

Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sem_data_proc_1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,label
0,Entire Facebook Staff Laughs As Man Tightens P...,1
1,Muslim Woman Denied Soda Can for Fear She Coul...,0
2,Bold Move: Hulu Has Announced That They’re Gon...,1
3,Despondent Jeff Bezos Realizes He’ll Have To W...,1
4,"For men looking for great single women, online...",1


### Word Tokenization

In [ ]:
! pip install spacy

In [ ]:
import spacy
! python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 59.0 MB/s eta 0:00:0000:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


#### Load Tokenizer for english language

In [8]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [9]:
import re


# Define text preprocessing function
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Remove special characters, numbers, and extra spaces
    text = re.sub(r'[^a-z\s]', '', text)

    # Process text with spaCy
    doc = nlp(text)

    # Tokenization, Lemmatization & Stopword Removal
    if remove_stopwords:
        tokens = [token.lemma_ for token in doc if not token.is_stop]
    else:
        tokens = [token.lemma_ for token in doc]
        

    return tokens  # Return list of tokens

# Apply preprocessing function
df['tokens'] = df['text'].apply(preprocess_text)

# Display results
print(df[['text', 'tokens']].head())

                                                text  \
0  Entire Facebook Staff Laughs As Man Tightens P...   
1  Muslim Woman Denied Soda Can for Fear She Coul...   
2  Bold Move: Hulu Has Announced That They’re Gon...   
3  Despondent Jeff Bezos Realizes He’ll Have To W...   
4  For men looking for great single women, online...   

                                              tokens  
0  [entire, facebook, staff, laugh, as, man, tigh...  
1  [muslim, woman, deny, soda, can, for, fear, sh...  
2  [bold, move, hulu, have, announce, that, they,...  
3  [despondent, jeff, bezos, realize, hell, have,...  
4  [for, man, look, for, great, single, woman, on...  


## Word2Vec
load google word2vec

finetune on onion dataset for better representation of satire specific context

In [10]:
custom_corpus = df['tokens'].tolist()  # Convert tokenized sentences into list format
custom_corpus

[['entire',
  'facebook',
  'staff',
  'laugh',
  'as',
  'man',
  'tighten',
  'privacy',
  'setting'],
 ['muslim',
  'woman',
  'deny',
  'soda',
  'can',
  'for',
  'fear',
  'she',
  'could',
  'use',
  'it',
  'as',
  'a',
  'weapon'],
 ['bold',
  'move',
  'hulu',
  'have',
  'announce',
  'that',
  'they',
  're',
  'go',
  'to',
  'go',
  'ahead',
  'and',
  'reboot',
  'shrill',
  'while',
  'its',
  'still',
  'on',
  'since',
  'you',
  'idiot',
  'will',
  'watch',
  'anything'],
 ['despondent',
  'jeff',
  'bezos',
  'realize',
  'hell',
  'have',
  'to',
  'work',
  'for',
  ' ',
  'second',
  'to',
  'earn',
  'back',
  'money',
  'he',
  'lose',
  'in',
  'divorce'],
 ['for',
  'man',
  'look',
  'for',
  'great',
  'single',
  'woman',
  'online',
  'dating',
  'offer',
  'a',
  'viable',
  'solution',
  'to',
  'the',
  'otherwise',
  'frustrating',
  'task',
  'of',
  'find',
  'longterm',
  'love',
  'arnsk'],
 ['kim',
  'jongun',
  'wonder',
  'if',
  'nuclear',
  

In [11]:
# Prepare the corpus for Word2Vec
custom_corpus = df['tokens'].tolist()  # Convert tokenized sentences into list format

In [ ]:
! conda install conda-forge::gensim  -y

Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [13]:
# Load pretrained embeddings
import gensim.downloader as api
pretrained_model = api.load('word2vec-google-news-300')

In [14]:
# Initialize Word2Vec model with same parameters as the pretrained model
word2vec = Word2Vec(vector_size=300, window=5, min_count=1, workers=4)

# First, build vocabulary using the custom dataset
word2vec.build_vocab(custom_corpus)

# Merge vocabulary with pretrained Word2Vec (avoid update=True to prevent overwriting)
pretrained_vocab = list(pretrained_model.key_to_index.keys())
word2vec.build_vocab([pretrained_vocab], update=True)

In [15]:
import numpy as np
# **Optimized Batch Weight Transfer**
common_words = list(set(word2vec.wv.key_to_index) & set(pretrained_model.key_to_index))

# Get indices of words in word2vec model
word_indices = [word2vec.wv.key_to_index[word] for word in common_words]

# Extract vectors in a single NumPy operation
word_vectors = np.array([pretrained_model[word] for word in common_words])

# Assign vectors in batch
word2vec.wv.vectors[word_indices] = word_vectors  # FAST batch assignment

In [16]:
# Fine-tune the model on the Onion dataset
word2vec.train(custom_corpus, total_examples=len(custom_corpus), epochs=5)

# Save the updated model -> diskspace
word2vec.save("fine_tuned_word2vec.model")

## LSTM Part

In [17]:
from gensim.models import Word2Vec

In [18]:
# Load the fine-tuned Word2Vec model -> diskspace
custom_corpus = df['tokens'].tolist()
word2vec = Word2Vec.load("fine_tuned_word2vec.model")

In [19]:
import numpy as np
import tensorflow as tf

2025-02-14 16:53:12.411104: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-14 16:53:12.425345: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739548392.443581  540062 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739548392.449483  540062 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-14 16:53:12.466639: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [27]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([" ".join(sentence) for sentence in custom_corpus])  # Fit tokenizer on text
word_index = tokenizer.word_index  # Dictionary mapping words to indices
vocab_size = len(word_index) + 1  # Adding 1 to account for padding index

In [28]:
# Convert text to sequences
sequences = tokenizer.texts_to_sequences([" ".join(sentence) for sentence in custom_corpus])
padded_sequences = pad_sequences(sequences, maxlen=50, padding='post', truncating='post')
labels = np.array(df["label"].values)

In [32]:
! conda install anaconda::scikit-learn -y
from sklearn.model_selection import train_test_split

Channels:
 - defaults
 - anaconda
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/sem_data_proc_1/miniconda3/envs/tf_env

  added / updated specs:
    - anaconda::scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-learn-1.6.1         |  py310h6a678d5_0        11.7 MB  anaconda
    ------------------------------------------------------------
                                           Total:        11.7 MB

The following NEW packages will be INSTALLED:

  scikit-learn       anaconda/linux-64::scikit-learn-1.6.1-py310h6a678d5_0 
  threadpoolctl      pkgs/main/linux-64::threadpoolctl-3.5.0-py310h2f386ee_0 



                                                                                
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [33]:
# **Train-Test Split**
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, labels, test_size=0.2, random_state=42, stratify=labels
)

In [ ]:
# Create embedding matrix
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if word in word2vec.wv:  # Check if word has pretrained embedding
        embedding_matrix[i] = word2vec.wv[word]  # Assign the vector

print(f"Embedding Matrix Shape: {embedding_matrix.shape}")  # Should be (vocab_size, 300)


Embedding Matrix Shape: (20459, 300)


LSTM Model

In [35]:
embedding_dim

300

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Define LSTM model
model = Sequential([
    Embedding(input_dim=vocab_size, 
              output_dim=embedding_dim, 
              weights=[embedding_matrix],  # Load pretrained embeddings
              input_length=50,  # Set sequence length
              trainable=False),  # Freeze embeddings (fine-tuned already)
    
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


/home/sem_data_proc_1/miniconda3/envs/tf_env/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [37]:
len(labels)

24000

In [38]:
padded_sequences.shape

(24000, 50)

In [39]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=2, validation_data=(X_test, y_test))

# Save the trained LSTM model
model.save("lstm_onion_model.h5")

Epoch 1/10
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 152s 16ms/step - accuracy: 0.6390 - loss: 0.6548 - val_accuracy: 0.6250 - val_loss: 0.6617
Epoch 2/10
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 149s 15ms/step - accuracy: 0.6238 - loss: 0.6639 - val_accuracy: 0.6250 - val_loss: 0.6546
Epoch 3/10
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 151s 16ms/step - accuracy: 0.6391 - loss: 0.6448 - val_accuracy: 0.7844 - val_loss: 0.5087
Epoch 4/10
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 153s 16ms/step - accuracy: 0.7951 - loss: 0.4490 - val_accuracy: 0.7710 - val_loss: 0.4687
Epoch 5/10
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 152s 16ms/step - accuracy: 0.8404 - loss: 0.3731 - val_accuracy: 0.8119 - val_loss: 0.4149
Epoch 6/10
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 152s 16ms/step - accuracy: 0.8704 - loss: 0.3069 - val_accuracy: 0.8263 - val_loss: 0.3915
Epoch 7/10
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 151s 16ms/step - accuracy: 0.9031 - loss: 0.2535 - val_accuracy: 0.8188 - val_loss: 0.4722
Epoch 8/10
9600/9600 ━━━━━━━━━━━━━━━━━━━━ 150s 16ms/step - accuracy: 

Apply LSTM and check metrics

In [45]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model

# Convert probabilities to binary labels (0 or 1) using a lambda function
threshold_fn = lambda y: (y > 0.5).astype(int)

# Load the trained LSTM model
lstm_model = load_model("lstm_onion_model.h5")
y_rfc_pred = threshold_fn(lstm_model.predict(X_test))

# Metrics
print('Accuracy: %.4f' % accuracy_score(y_test, y_rfc_pred))
print(confusion_matrix(y_test, y_rfc_pred))
print(classification_report(y_test,y_rfc_pred))

150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Accuracy: 0.8279
[[2554  446]
 [ 380 1420]]
              precision    recall  f1-score   support

           0       0.87      0.85      0.86      3000
           1       0.76      0.79      0.77      1800

    accuracy                           0.83      4800
   macro avg       0.82      0.82      0.82      4800
weighted avg       0.83      0.83      0.83      4800

